In [ ]:
# загружаем данные
import pandas as pd
import numpy as np

ratings = pd.read_csv('data_ml-1m/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [ ]:
# создаём разреженную матрицу user*item
from scipy.sparse import coo_matrix

user_item_matrix = coo_matrix((
    (ratings["rating"] > 0).astype(np.float32), # по колонке оценок пораждается булевская колонка "нравится"
    (ratings["userId"], ratings["movieId"])    # назначение матрицы строк и столбцов (ID пронумерованны плотно)
))

user_item_matrix.eliminate_zeros()

In [ ]:
# делим разреженную матрицу на обучающую и тестовую
total_len = user_item_matrix.data.size
train_len = int(total_len * 0.8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace=False)
train_mask = np.in1d(all_indices, train_indices)

In [ ]:
# функция наложения маски на разреженные матрицы
def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            (arr.row[mask], arr.col[mask])
        ), arr.shape
    )

In [ ]:
# подготовка train и  test матриц

train_csr = get_masked(user_item_matrix, train_mask).tocsr()
train = train_csr.T
test_coo = get_masked(user_item_matrix, ~train_mask)
test_csr = test_coo.tocsr()

In [ ]:
# получение рекомендаций
users = list(set(test_coo.row))

def get_recs(user, model):
    return{
        user: model.recommend(userid=user, user_items=train_csr, N=50)
        for user in users
    }

In [ ]:
# подсчет hitrate 
def hitrate (k, recs, users):
    hits = 0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user]) # выемка рекомендаций, если они есть из матрицы 
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
    return hits / len(users) 

In [ ]:
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares

In [ ]:
# крсинусная мера
cosine = CosineRecommender(K=22)

In [ ]:
%%time

cosine.fit(train)


CPU times: user 825 ms, sys: 28.1 ms, total: 853 ms
Wall time: 847 ms


In [ ]:
%%time

# посчитаем по N_test рекомендаций для каждого пользователя из тестовой выборки
cosine_recs = get_recs(users, cosine)

CPU times: user 397 ms, sys: 19.8 ms, total: 417 ms
Wall time: 418 ms


In [ ]:
print('hitrate=50  ', hitrate(50, cosine_recs, users))

hitrate=50   0.9671805072103431


In [ ]:
# ALS
import os

os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'
als = AlternatingLeastSquares(factors=60, iterations=20, random_state=42)

In [ ]:
%%time

als.fit(train)


CPU times: user 22.9 s, sys: 205 ms, total: 23.1 s
Wall time: 5.86 s


In [ ]:
%%time

# посчитаем по N_test рекомендаций для каждого пользователя из тестовой выборки
als_recs = get_recs(users, als)

CPU times: user 9.76 s, sys: 47.2 ms, total: 9.81 s
Wall time: 2.45 s


In [ ]:
print('hitrate=50  ', hitrate(50, als_recs, users))

hitrate=50   0.9799436432952097


In [ ]:
# нормализация score
from sklearn.preprocessing import normalize

def normalize_score(alg, users):
    for user in users:
        if alg[user]:
            rec_items, rec_us = zip(*alg[user]) # выемка рекомендаций, если они есть из матрицы
            rec_us = normalize(np.array(rec_us, dtype=object).reshape(1, -1))
            rec_us = rec_us[0].tolist()
            for i in range(len(alg[user])):
                alg[user][i] = (rec_items[i], rec_us[i])
    return alg

In [ ]:
als_recs = normalize_score(als_recs, users)
cosine_recs = normalize_score(cosine_recs, users)

In [ ]:
# поиск одинаковых фильмов в рекомендациях для одного пользователя
def grouped_movies(movies):
    result = {}
    for movie in movies:
        # проверка, встречался ли уже этот фильм
        if movie[0] in result:
            # добавление текушего значения score
            result[movie[0]] = result[movie[0]] + movie[1]
        else:
            result[movie[0]] = movie[1]
    return result

In [ ]:
# создание словаря рекомендаций на основании взвешенных score
new_recs = dict()
for user in users:
    for i in range(len(als_recs[user])):
        # так как ALS работает лучше, коэф = 0.7
        als_recs[user][i] =  (als_recs[user][i][0], 0.7*als_recs[user][i][1])
    for i in range(len(cosine_recs[user])):
        # коэф = 1 - коэф для ALS
        cosine_recs[user][i] =  (cosine_recs[user][i][0], 0.3*cosine_recs[user][i][1])     
    new_rec = als_recs[user] + cosine_recs[user]
    new_rec = grouped_movies(new_rec)
    new_recs[user] = dict(sorted(new_rec.items(), key=lambda item: item[1], reverse = True ))
    new_recs[user] = [(movie_id, score) for movie_id, score in new_recs[user].items()] 

In [ ]:
print('ALS hitrate:  ', hitrate(50, als_recs, users))
print('Cosine hitrate:  ', hitrate(50, cosine_recs, users))
print('Hybrid hitrate:  ', hitrate(50, new_recs, users))

ALS hitrate:   0.9799436432952097
Cosine hitrate:   0.9671805072103431
Hybrid hitrate:   0.9801093983092989
